In [12]:
with open("example_md_to_text.txt", "r", encoding="latin-1") as f: # TO DO: check proper encoding for .md files
    markdown_example = f.read()

with open("example.pdf", "rb") as f:
    pdf = f.read()

with open("HF_TOKEN.txt", "r") as f:
    hf_token = f.read()

with open("GROQ_KEY.txt", "r") as f:
    groq_token = f.read()

In [13]:
from huggingface_hub import HfFolder, whoami

HfFolder.save_token(hf_token)
print(whoami()["name"])

alberto-lorente


In [14]:
import torch

device = "cpu"
if torch.cuda.is_available():
    print("Cuda available")
    device = torch.device('cuda')

Cuda available


In [15]:
pdf_path = "example.pdf"

base_prompt="""CONTEXTE
L'image suivante contient une page et un tableau. 

TÂCHE
Décrivez le tableau et le contenu de la page en accordant une attention particulière au contexte qui l'entoure, qu'il s'agisse de budgets, de dates, d'élections, d'agendas, de projets futurs ou de sujets connexes. 

FORMAT DE LA RÉPONSE
Votre réponse doit être aussi détaillée que possible. 
Votre résultat doit être la description du tableau directement.
La langue de votre réponse est le français"""


summary_prompt = """Il s'agit d'un texte concernant un projet de géothermie :
TEXTE: 
{}

TÂCHE:
Make a one page summary paying special attention to all the administrative matters like budgets, plans, actions to take in the future, organizational and hierarchical charts, announcements, meetings, contacts, elections, reports and all the related topics to these.

OUTPUT:
Publier directement le résumé.
"""


In [16]:
from council_rag.preprocessing import preprocess_markdown_text
from council_rag.data_transformations import process_tables, summarize_clusters
import time

start = time.time()

paragraphs_list, clusters_dict = preprocess_markdown_text(markdown_example,
                                                        model_id ="Snowflake/snowflake-arctic-embed-m-long", 
                                                        spacy_model="fr_core_news_sm", 
                                                        n_sents_per_para=10,
                                                        device=device
)

processed_tables = process_tables(pdf_path, 
                                base_prompt, 
                                groq_token)

clusters_dict = summarize_clusters(clusters_dict, 
                                    summary_prompt, 
                                    groq_token, 
                                    model="gemma2-9b-it", 
                                    token_limit=14000, 
                                    sleep_time=60)

end = time.time()
print((end - start)/60)

ValueError: Loading Snowflake/snowflake-arctic-embed-m-long requires you to execute the configuration file in that repo on your local machine. Make sure you have read the code there to avoid malicious use, then set the option `trust_remote_code=True` to remove this error.

The variables we are going to query are:

In [7]:
# paragraphs_dict 
# processed_tables
# clusters_dict

## Indexing and querying

Deleting some info that we won't need and turning the cluster dict into a list to match the paragraphs list.

In [8]:
del paragraphs_list[0]["para_embedding"]

In [9]:
clusters_list = list(clusters_dict.values())
i = 0
while i < len(clusters_list):
    clusters_list[i]["cluster"] = i
    i += 1


Getting the embedding model and setting up the index and vector stores.

In [10]:
from sentence_transformers import SentenceTransformer


embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

In [11]:
# checking the shape of the embedding to pass it to the index
shape_emb = embedding_model.encode("Hello World!")
emd_dims =  shape_emb.shape[0]

In [12]:
import faiss

# init the intex
index = faiss.IndexFlatL2(emd_dims)

In [ ]:
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document

# creating the vector store
vector_store = FAISS(embedding_model, 
                    index, 
                    InMemoryDocstore({}), 
                    {})

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [14]:
clusters_list[0].keys()

dict_keys(['para_indexes', 'union_paras', 'cluster_summary', 'cluster'])

Processing all the information we have collected so far into Document objects that will be fed into the vector store.

In [15]:
cluster_paras = [] # to process into smaller chunks
cluster_summ_docs = [] # filled with the cluster summaries langchain doc type

for cluster in clusters_list:
    
    cluster_para = cluster["union_paras"]
    cluster_summ = cluster["cluster_summary"]
    
    cluster_paras.append(cluster_para)
    cluster_summ_docs.append(Document(page_content=cluster_summ, metadata={"cluster": cluster["cluster"],
                                                                            "type": "summary"}))

In [16]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=450, 
                                        chunk_overlap=35,
                                        length_function=len,
                                        is_separator_regex=False)

In [17]:
cluster_paras_docs = []
for cluster_union in cluster_paras:
    cluster_union_docs = splitter.split_text(cluster_union)
    cluster_paras_docs.append(cluster_union_docs)

In [18]:
len(cluster_paras_docs[0])

54

In [19]:
cluster_chunks = []
i = 0
while i < len(cluster_paras_docs):  
    curr_cluster = cluster_paras_docs[i]
    for cluster_chunk in curr_cluster:
        cluster_chunks.append(Document(page_content=cluster_chunk, metadata={"cluster": i,
                                                                                "type": "cluster_chunk"}))
    i += 1


In [20]:
augmented_table_chunks = []
table_descriptions_chunks = []
html_tables_chunks = []

for table in processed_tables:
    augmented_chunk = table["table_augmented_context"]
    augmented_table_chunks.append(Document(page_content=augmented_chunk, metadata={"type": "augmented_table"}))
    
    description_chunk = table["table_context"]
    table_descriptions_chunks.append(Document(page_content=description_chunk, metadata={"type": "description_table"}))
    
    html_chunk = table["table_html"]
    html_tables_chunks.append(Document(page_content=html_chunk, metadata={"type": "html_table"}))
    


Overall we have:

- The individual text chunks.
- The clusters summaries chunks.
- The augmented table chunks.
- The table descriptions chunks.
- The html tables chunks.


In [21]:
cluster_chunks[0]

Document(metadata={'cluster': 0, 'type': 'cluster_chunk'}, page_content="PROCES VERBAL\n\n\n\n L'an deux mille vingt et un, le 28 octobre à 18 h 00, les conseillers communautaires de Roannais Agglomération, se sont réunis à l\x92Espace Chorum \x96 Halle Vacheresse \x96 Rue des Vernes à Roanne.\n\n\n\n La convocation de tous les conseillers a été faite le 22 octobre 2021, dans les formes et délais prescrits par la loi, par Yves Nicolin, Président.\n\n\n\n # Etaient présents :")

In [22]:
cluster_summ_docs[0]

Document(metadata={'cluster': 0, 'type': 'summary'}, page_content='## Summary of Roannais Agglomération meeting minutes\n\nThis document summarizes prominent decisions made by the Roannais Agglomération council meeting on October 28, \n\n\n**Administrative Matters:**\n\nThe meeting was convened on October \n\n**Appoval of the Budget modificative \n\n\n* Budget allocation for "Réorganisation collecte déchets ménagère"\n\n    \n* Adjustment of contributions\n    \n\n\n**Meetings and Participatinate\n\nThe meeting was attended by numerous councilors \n**Organization**\n\n* \n\n*   Rôle de \n* Débat:\n**Actions to be taken\nThe council\n\n\n **Future Plans\n *  \n\n\n** Reports\n*\n **Contact information\n\n**Elections\n*   \n**Announcement\n\n##\n\nThis document summarizes the main points discussed at a meeting of the Roannais Agglomération council on  \n\n* \n\n*\n\n\n**Decisions:\n\n\n**Participants:**\n\n* Chair: Yves Nicolin\n\n\nThis document provides a summary of  \n\n\n**Budget**:\

In [23]:
augmented_table_chunks[0]

Document(metadata={'type': 'augmented_table'}, page_content='Bien sûr, je suis là pour vous aider au mieux. Sur cette nouvelle photo du questionnaire, le tableau et la page ainsi que la légende proposent des informations semblables à celles que nous ayons connus sur le monde politique.\n\nBien sur, il faut porter une attention particulière sur l\'élection, qui ne se présentait pas sous son nom dans notre précédent question. \nContrairement à cela, les images ci-dessus sont en français, et ce qui peut marquer le contexte politique à l\'heure actuelle est l\'élection présidentielle, comme on peut le voir dans le lien ci-dessous qui commente l\'élection du Président de la République française du 24 avril 2022 (Paris).\n\nC’est en effet la troisième, en moins d’un an. Il s’agit donc d’un scrutin d’exception. À commencer, ce n’est pas trop clair si la France est toujours respectera le fonctionnement de la constitution, qui donnait le droit à un mandat uniquement de cinq ans.\n\nDans notre c

In [24]:
table_descriptions_chunks[0]

Document(metadata={'type': 'description_table'}, page_content="Bien sûr, je suis là pour vous aider au mieux. Sur cette nouvelle photo du questionnaire, le tableau et la page ainsi que la légende proposent des informations semblables à celles que nous ayons connus sur le monde politique.\n\nBien sur, il faut porter une attention particulière sur l'élection, qui ne se présentait pas sous son nom dans notre précédent question. \nContrairement à cela, les images ci-dessus sont en français, et ce qui peut marquer le contexte politique à l'heure actuelle est l'élection présidentielle, comme on peut le voir dans le lien ci-dessous qui commente l'élection du Président de la République française du 24 avril 2022 (Paris).\n\nC’est en effet la troisième, en moins d’un an. Il s’agit donc d’un scrutin d’exception. À commencer, ce n’est pas trop clair si la France est toujours respectera le fonctionnement de la constitution, qui donnait le droit à un mandat uniquement de cinq ans.\n\nDans notre cas

In [25]:
html_tables_chunks[0]

Document(metadata={'type': 'html_table'}, page_content='Bien sûr, je suis là pour vous aider au mieux. Sur cette nouvelle photo du questionnaire, le tableau et la page ainsi que la légende proposent des informations semblables à celles que nous ayons connus sur le monde politique.\n\nBien sur, il faut porter une attention particulière sur l\'élection, qui ne se présentait pas sous son nom dans notre précédent question. \nContrairement à cela, les images ci-dessus sont en français, et ce qui peut marquer le contexte politique à l\'heure actuelle est l\'élection présidentielle, comme on peut le voir dans le lien ci-dessous qui commente l\'élection du Président de la République française du 24 avril 2022 (Paris).\n\nC’est en effet la troisième, en moins d’un an. Il s’agit donc d’un scrutin d’exception. À commencer, ce n’est pas trop clair si la France est toujours respectera le fonctionnement de la constitution, qui donnait le droit à un mandat uniquement de cinq ans.\n\nDans notre cas, l

In [40]:
all_docs = cluster_chunks + cluster_summ_docs + augmented_table_chunks + table_descriptions_chunks + html_tables_chunks

In [41]:
len(all_docs)

286

In [ ]:
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(all_docs))]

In [43]:
# for doc in all_docs:
#     print(type(doc))
#     print(doc.metadata)
#     print(doc.page_content)

In [45]:
# help(vector_store.add_documents)

In [48]:
cluster_chunks[0]

Document(metadata={'cluster': 0, 'type': 'cluster_chunk'}, page_content="PROCES VERBAL\n\n\n\n L'an deux mille vingt et un, le 28 octobre à 18 h 00, les conseillers communautaires de Roannais Agglomération, se sont réunis à l\x92Espace Chorum \x96 Halle Vacheresse \x96 Rue des Vernes à Roanne.\n\n\n\n La convocation de tous les conseillers a été faite le 22 octobre 2021, dans les formes et délais prescrits par la loi, par Yves Nicolin, Président.\n\n\n\n # Etaient présents :")

In [49]:
uuids[0]

'a76d7734-06a5-4b04-adc6-e139860bbcac'

In [1]:
# adding the documents to the vectorstore


# vector_store.add_documents(documents=cluster_chunks, ids=uuids)
# if this still does not work, we can do FAISS.from_texts with the embeddings computed manually

UP UNTIL THIS POINT, EVERYTHING IS PACKAGED MISSING THE ACTUAL POPULATING OF THE DB

Querying

In [ ]:
query = "Get from Tunji's thingie"

results = vector_store.similarity_search(
    query,
    k=2,
    filter={"source": "tweet"}, # put the clustering, etc here
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]") # this is how to get the attributes

CHECK WITH THE FIXED VECTOR STORE BEFORE PACKAGING INTO COUNCIL_RAG.RAG AND MAIN.PY

In [7]:
# algo to query the store in the right order
def query_vector_store(vector_store, query):
        # first search through the summaries
        results_query_level_one = vector_store.similarity_search(
                                                                query,
                                                                k=1,
                                                                filter={"type": "summary"}, 
                                                                )

        cluster_level_one = results_query_level_one[0].metadata["cluster"]

        # now we search only in the cluster retrieved in the first step
        results_query_level_two = vector_store.similarity_search(
                                                        query,
                                                        k=5,
                                                        filter={"cluster": cluster_level_one,
                                                                "type": "cluster_chunk"}, 
                                                        )
        results_query_tables = vector_store.similarity_search(
                                                        query,
                                                        k=1,
                                                        filter={"type": 'description_table'}, # {"$in": []} 
                                                        )
        
        relevant_facts = [sent_query_level_two.page_content for sent_query_level_two in results_query_level_two]
        relevant_table = results_query_tables.page_content
        
        return relevant_facts, relevant_table

In [8]:
def augment_query_rag(query, relevant_facts, relevant_table):
    
    aug_prompt="""\nVoici quelques faits pertinents pour vous aider à répondre et una description du qui peut s'avérer utile. 
    Si la description du n'est pas utile, ignorez-le.\n"""
    
    augmented_data_string = aug_prompt + "\n".join(relevant_facts) + "\n" + relevant_table
    
    formated_augmented_query = query.format(augmented_data_string) # the original query has to have a format string
    # formated_augmented_query = query + augmented_data_string       # if that is not the case, we can do this instead
    
    return formated_augmented_query

In [9]:
from groq import Groq

def final_query(query, model, groq_key):
    
    client = Groq(api_key=groq_key)
    
    messages = [
                {"role": "system",
                "content":  "Vous êtes un assistant utile" },
                {"role": "user",
                "content":  query}
                ]
    
    chat_completion = client.chat.completions.create(messages=messages, model=model)
    return chat_completion.choices[0].message.content

In [10]:
# final fx to query our list of queries

def query_multiple(list_of_queries, vector_store, model, groq_key):
    
    responses = []
    for query in list_of_queries:
        
        relevant_facts, relevant_table = query_vector_store(vector_store, query)
        formated_augmented_query = augment_query_rag(query, relevant_facts, relevant_table)
        response = final_query(formated_augmented_query, model, groq_key)
        responses.append(response)
    
    return responses